In [2]:
import toolz as tz


In [3]:
class Container:
    def __init__(self, x):
        self.__value__ = x

    @classmethod
    def of(cls, x):
        return cls(x)

    def map(self, f):
        return Container.of(f(self.__value__))

    def __repr__(self):
        return str(self.__value__)


class Maybe:
    def __init__(self, x):
        self.__value__ = x
        self.isNothing = self.__value__ is None

    @classmethod
    def of(cls, x):
        return cls(x)

    def map(self, f):
        return self if self.isNothing else Maybe.of(f(self.__value__))

    def __repr__(self):
        return "Nothing" if self.isNothing else f"Just({self.__value__})"


@tz.curry
def prop(name, obj):
    return obj[name]


@tz.curry
def map_(f, functor):
    return functor.map(f)

@tz.curry
def maybe(v, f, m):
    return v if m.isNothing else f(m.__value__)

In [4]:
Container.of(3).map(lambda two: two + 2)


5

In [5]:
Maybe.of('abc').map(len), Maybe.of('').map(len)

(Just(3), Just(0))

In [6]:
map_(len, Container.of('abc'))

3

In [7]:
@tz.curry
def withdraw(amount, balance):
    return Maybe.of({"balance": balance - amount} if balance >= amount else None)

def updateLedger(account):
    return account

def remainingBalance(balance):
    return f'Your balance is ${balance}'

finishTransaction = tz.compose(
    remainingBalance,
    updateLedger
)

getTwenty = tz.compose(
    maybe(f'You\'re broke!', finishTransaction),
    withdraw(20),
    prop('balance')
)

getTwenty({"balance": 100}), getTwenty({"balance": 10})

("Your balance is ${'balance': 80}", "You're broke!")